In [2]:
import datos_jugadores

# Descargar jugadores de La Liga (2024-2025)

## Obtención de los enlaces de las páginas de todos los jugadores de La Liga

In [2]:
lista_jugadores = datos_jugadores.obtenerEnlacesJugadores()
lista_jugadores

['/jugador/julen-agirrezabala',
 '/jugador/unai-simon',
 '/jugador/de-marcos',
 '/jugador/yuri-b',
 '/jugador/gorosabel',
 '/jugador/vivian',
 '/jugador/i-lekue',
 '/jugador/paredes-2',
 '/jugador/yeray-2',
 '/jugador/vesga',
 '/jugador/ander',
 '/jugador/o-sancet-1',
 '/jugador/r-de-galarreta',
 '/jugador/benat-prados',
 '/jugador/berenguer',
 '/jugador/alvaro-djalo',
 '/jugador/guruzeta',
 '/jugador/williams-jr',
 '/jugador/malcom-adu-ares',
 '/jugador/williams',
 '/jugador/javier-marton',
 '/jugador/nicolas-serrano',
 '/jugador/horatiu-moldovan',
 '/jugador/oblak',
 '/jugador/simeone',
 '/jugador/cesar-azpilicueta',
 '/jugador/reinildo',
 '/jugador/nahuel-molina',
 '/jugador/j-m-gimenez',
 '/jugador/santiago-mourino',
 '/jugador/axel-witsel',
 '/jugador/de-paul',
 '/jugador/arthur-vermeeren',
 '/jugador/pablo-barrios-rivas',
 '/jugador/riquelme',
 '/jugador/koke',
 '/jugador/m-llorente',
 '/jugador/lemar',
 '/jugador/javi-galan',
 '/jugador/samuel-lino',
 '/jugador/griezmann',
 '/ju

## Obtención de los datos de todos los jugadores de La Liga

In [4]:
df_jugadores = datos_jugadores.obtenerDatosJugadores(lista_jugadores)

In [5]:
# Tratamiento de los datos
df_jugadores['fecha'] = df_jugadores.fecha.str.slice(start=0, stop=10)
df_jugadores

,jugador,apodo,fecha,peso,altura,pais,posicion,equipo,dorsal,url
0,Julen Agirrezabala,Agirrezabala,2000-12-26,80,187,ES,Portero,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p470...
1,Unai Simón,Unai Simon,1997-06-11,88,190,ES,Portero,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p212...
2,Óscar De Marcos,De Marcos,1989-04-14,77,182,ES,Defensa,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p590...
3,Yuri Berchiche,Yuri,1990-02-10,79,181,ES,Defensa,Athletic Club,,https://assets.laliga.com/squad/2024/t174/p426...
4,Andoni Gorosabel,Gorosabel,1996-08-04,70,174,ES,Defensa,Athletic Club,,
...,...,...,...,...,...,...,...,...,...,...
345,Santi Comesaña,Santi C.v.,1996-10-05,75,184,ES,Centrocampista,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p230...
346,Gerard Moreno,Gerard,1992-04-07,77,180,ES,Delantero,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p937...
347,Alfonso Pedraza,A. Pedraza,1996-04-09,73,184,ES,Delantero,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p208...
348,Yeremy Pino,Yeremy,2002-10-20,65,172,ES,Delantero,Villarreal Club de Fútbol SAD,,https://assets.laliga.com/squad/2024/t449/p488...


In [6]:
# Almacenamiento de los datos en un Excel
df_jugadores.to_excel('info_jugadores_la_liga.xlsx', index=False)   

# Descargar jugadores de la Premier League (2024-2025)

In [3]:
enlaces_equipos_pl = datos_jugadores.obtenerEnlacesEquiposPremier()
enlaces_equipos_pl

['/clubs/1/Arsenal/squad',
 '/clubs/2/Aston-Villa/squad',
 '/clubs/127/Bournemouth/squad',
 '/clubs/130/Brentford/squad',
 '/clubs/131/Brighton-and-Hove-Albion/squad',
 '/clubs/4/Chelsea/squad',
 '/clubs/6/Crystal-Palace/squad',
 '/clubs/7/Everton/squad',
 '/clubs/34/Fulham/squad',
 '/clubs/8/Ipswich-Town/squad',
 '/clubs/26/Leicester-City/squad',
 '/clubs/10/Liverpool/squad',
 '/clubs/11/Manchester-City/squad',
 '/clubs/12/Manchester-United/squad',
 '/clubs/23/Newcastle-United/squad',
 '/clubs/15/Nottingham-Forest/squad',
 '/clubs/20/Southampton/squad',
 '/clubs/21/Tottenham-Hotspur/squad',
 '/clubs/25/West-Ham-United/squad',
 '/clubs/38/Wolverhampton-Wanderers/squad']

## Obtención de los datos de todos los jugadores de la Premier League

In [5]:
df_jugadores_pl = datos_jugadores.obtenerDatosJugadoresPremier()
df_jugadores_pl

,jugador,equipo,dorsal,posicion,pais,url
0,Aaron Ramsdale,Arsenal,1,Goalkeeper,England,https://resources.premierleague.com/premierlea...
1,Karl Hein,Arsenal,31,Goalkeeper,Estonia,https://resources.premierleague.com/premierlea...
2,David Raya,Arsenal,22,Goalkeeper,Spain,https://resources.premierleague.com/premierlea...
3,Tommy Setford,Arsenal,36,Goalkeeper,England,https://resources.premierleague.com/premierlea...
4,Lucas Martin,Arsenal,54,Goalkeeper,Denmark,https://resources.premierleague.com/premierlea...
...,...,...,...,...,...,...
721,Emilio Ballard-Matthews,Wolverhampton Wanderers,78,Forward,England,https://resources.premierleague.com/premierlea...
722,Leon Chiwome,Wolverhampton Wanderers,84,Forward,England,https://resources.premierleague.com/premierlea...
723,Rodrigo Gomes,Wolverhampton Wanderers,,Forward,Portugal,https://resources.premierleague.com/premierlea...
724,Chiquinho,Wolverhampton Wanderers,20,Forward,Portugal,https://resources.premierleague.com/premierlea...


In [6]:
# Almacenamiento de los datos en un Excel
df_jugadores_pl.to_excel('info_jugadores_premier_league.xlsx', index=False)   